Test


In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from math import *
import time
import copy
import random
from matplotlib import animation
from IPython.display import HTML
import functions

%matplotlib

Using matplotlib backend: <object object at 0x000002B34B645500>


In [3]:
#img = Image.open('D:\Dump2\Phys 437\co-culture codes\deconv_s1t01c1-3.jpg')
#img.show()
#img.size

In [4]:
### RESETTING IMAGE RESOLUTION

umpp = 0.91 # um per pixel
time_i = 0 # initial time frame
time_f = 23 # final time frame
#path = "Data/nov3_pad3_YFP/monoculture_nov3_pad3_22hr-02_s2t" # file name until timestamp
fc = "Cy3" # fluorescent channel name

nump = 50 # number of pixels to combine when resizing
dx = nump*umpp

dthr = 1 # time interval between frames in hr

# Import images

Phase = []
FC = []

t = time_i+1

im = np.array(Image.open(path+str(t).zfill(2)+".jpg").convert('L'))
Ly = len(im)
Lx = len(im[0])

Ly_new = int(Ly/nump)
Lx_new = int(Lx/nump)

size = Lx_new, Ly_new

def resolve_down(im, factor):
    Lx, Ly = im.size
    
    LR_off = Lx%factor
    L_off = int(LR_off/2)
    R_off = Lx-LR_off+L_off
    TB_off = Ly%factor
    T_off = int(TB_off/2)
    B_off = Ly-TB_off+T_off
    
    im1 = np.array(im.crop((L_off, T_off, R_off, B_off)))
    
    new = []
    for a in im1:
        b = a.reshape(-1, factor).mean(axis=1)
        new.append(b)
    new = np.array(new).T
    neww = []
    for a in new:
        b = a.reshape(-1, factor).mean(axis=1)
        neww.append(b)
    newww = np.array(neww).T
    
    return newww

while t<time_f:
    imp = Image.open(path+str(t).zfill(2)+"_Phase.jpg").convert('L')
    imp_resized = resolve_down(imp, nump)
    #imp_resized = np.array(imp.resize(size, Image.ANTIALIAS)).astype(np.float32)/
    imf = Image.open(path+str(t).zfill(2)+"_{}.jpg".format(fc)).convert('L')
    imf_resized = resolve_down(imf, nump)
    #imf_resized = np.array(imf.resize(size, Image.ANTIALIAS)).astype(np.float32)
    
    Phase.append(imp_resized)
    FC.append(imf_resized)
    t+=1
    
Phase = np.array(Phase)
FC = np.array(FC)

FileNotFoundError: [Errno 2] No such file or directory: 'deconv_s1t01.jpg'

In [14]:
import functions
path, fc1, fc2, time_i, time_f, nump, umpp, thresh, dthr, sd_min = 'deconv_s1t', 'CFP','Cy3',1,40,50,0.91,1,1,0.5 
input = path, fc1, fc2, time_i, time_f, nump, umpp, thresh, dthr, sd_min
DConData = functions.read_data_as_array(input)
DConData
#a = functions.resolve_down(img, 2)

(array([[[ 27.0104,   6.016 ,   5.9208, ...,  19.6632,  18.8772,
           16.2676],
         [  6.6728,   6.7788,  81.3636, ...,  20.2736,  19.256 ,
           16.4064],
         [ 30.9676,  51.5576,  78.3736, ...,  21.2596,  20.388 ,
           16.83  ],
         ...,
         [ 22.6144,  23.9104,  23.7088, ...,   1.8188,   2.7324,
            1.4356],
         [ 24.4008,  24.7876,  24.978 , ...,   3.0488,   3.0784,
            2.258 ],
         [ 28.7816,  25.1964,  24.5784, ...,   2.468 ,   2.5576,
            3.3812]],
 
        [[205.7848,  42.3768,   7.9652, ...,  19.2564,  18.3656,
           15.784 ],
         [ 23.2332,  47.5652, 107.0388, ...,  19.8444,  18.8936,
           16.0112],
         [244.1816, 235.258 , 245.1724, ...,  20.86  ,  19.968 ,
           16.4184],
         ...,
         [ 22.1212,  23.3476,  23.15  , ...,   1.5588,   2.1108,
            1.3472],
         [ 23.5984,  24.0424,  24.1628, ...,   2.374 ,   2.4948,
            1.662 ],
         [ 28.6212,  24

In [ ]:
plt.imshow(functions.read_data_as_array(input))

TypeError: Invalid shape (3, 38, 161, 176) for image data

In [ ]:
## GRADIENT FUNCTIONS

def grad_vec_field(series, t, Lx, Ly, dx):

    xr = np.arange(0, Ly*dx, dx)
    yr = np.arange(0, Lx*dx, dx)

    # Create grid corresponding to xr and yr arrays
    xx, yy = np.meshgrid (xr, yr, indexing = 'ij')

    zz1 = series[t+1]
    gradx1, grady1 = np.gradient(zz1, dx, dx)

    zz0 = series[t]
    gradx0, grady0 = np.gradient (zz0, dx, dx)
    
    return(xx, yy, gradx1-gradx0, grady1-grady0)

def plot_vec_field(series, t, Lx, Ly, dx):
    
    xx, yy, Dx, Dy = grad_vec_field(series, t, Lx, Ly, dx)

    fig = plt.figure(constrained_layout=True, figsize=(15,8))
    axs = fig.subplot_mosaic([['Left', 'TopRight'],['Left', 'BottomRight']],
                          gridspec_kw={'width_ratios':[2, 1]})
    
    TB = axs['TopRight'].imshow(-Dx, cmap='coolwarm')
    axs['TopRight'].set_title('Y component of ascent rate')
    fig.colorbar(TB, ax=axs['TopRight'])
    
    LR = axs['BottomRight'].imshow(Dy, cmap='coolwarm')
    axs['BottomRight'].set_title('X component of ascent rate')
    fig.colorbar(LR, ax=axs['BottomRight'])
    
    skip = (slice(None, None, 2), slice(None, None, 2))

    axs['Left'].quiver(yy[skip], np.flip(xx)[skip], Dy[skip] , -Dx[skip],
                      color='black', headwidth=5, scale=1, headlength=5)

## MODEL FUNCTIONS

def nbr_sum(M):
    """Get the discrete Laplacian of matrix M"""
    L = np.roll(M, (0,-1), (0,1)) # right neighbor
    L += np.roll(M, (0,+1), (0,1)) # left neighbor
    L += np.roll(M, (-1,0), (0,1)) # top neighbor
    L += np.roll(M, (+1,0), (0,1)) # bottom neighbor
    
    return L

def discrete_laplacian(M):
    """Get the discrete Laplacian of matrix M"""
    L = -4*M
    L += nbr_sum(M)
    
    return L

def divide(M1,M2):
    """To divide two numpy arrays M1/M2 element-wise and avoide division by zero"""
    f = np.divide(M1, M2, out=np.zeros_like(M1), where=M2!=0)
    return f

def update(C, S, Parameters):
    
    k, mu, Rhf, Qhf, Phf, Y, Ks, DC, DS = tuple(Parameters)
    
    R = C/(Rhf+C)
    Q = C/(Qhf+C)
    P = Phf/(Phf+C)
    
    Sdiff = res*DS*discrete_laplacian(S) - dt*mu*C*divide(S,Y*(Ks+S))
    Cdiff = dt*mu*(1-R)*C*S/(Ks+S) + res*DC*mu*(P*nbr_sum(C*Q*S/(Ks+S)) - Q*C*S*nbr_sum(P)/(Ks+S))
    
    C += Cdiff
    S += Sdiff
    # Remove negative values
    S[S < 0] = 0 
    
    # Reflecting boundaries
    
    C[0,:], S[0,:] = C[1,:], S[1,:]          # top
    C[-1,:], S[-1,:] = C[-2,:], S[-2,:] # bottom
    C[:,0], S[:,0] = C[:,1], S[:,1]         # left
    C[:,-1], S[:,-1] = C[:,-2], S[:,-2]  # right
                            
    return C, S

def get_error(Phase, FC, Parameters):
    
    k = Parameters[0]
    # Initialize
    C = copy.deepcopy(FC[0])*k
    C -= threshold
    C[C<0]=0
    S = np.ones((Ly_new,Lx_new))

    # Simulate

    update_every = dthr*3600 # number of time steps after which data is stored
    C_time = []
    S_time = []

    start = time.time()

    for tt in range(T):
        if tt%update_every == 0:
            C_time.append(C.copy())
            S_time.append(S.copy())
        C, S = update(C, S, Parameters)

    stop = time.time()

    error_int = 0
    error_grad = 0

    t = time_i

    while t<time_f-2:
        xx, yy, DxO, DyO = grad_vec_field(Phase, t, Lx_new, Ly_new, dx)
        xx, yy, DxP, DyP = grad_vec_field(C_time, t, Lx_new, Ly_new, dx)
    
        error_int += np.sum((C_time[t]-Phase[t])**2)
        error_grad += np.sum((DxP-DxO)**2 + (DyP-DyO)**2)
        t+=1

    return(error_int, error_grad)

def cap_par(Parameters, idx_ex):
    i = 0
    for par in Parameters:
        if i in idx_ex:
            continue
        else:
            Parameters[i] = max(0, par)
        i+=1
    return(Parameters)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import study_init as st
# Set simulation parameters
mu1 = 0.18 / 3600
mu2 = 0.18 / 3600
Rhf = 180
Qhf = 40
Phf = 100
Y = 40
Ks = 0.001
DC = 2200
DS = 120
DA1 = 100
DA2 = 100
IC1 = 0.5
IC2 = 0.5
n1 = 1
n2 = 4
Km1 = 0.1
Km2 = 0.1
V1 = 0.001
V2 = 0.001

model_parameters = np.array([mu1, mu2, Rhf, Qhf, Phf, Y, Ks, DC, DS, DA1, DA2, IC1, IC2, n1, n2, Km1, Km2, V1, V2])

r = 5
I0 = 10
f1 = 0.5
Lx, Ly = 82, 102

init_parameters = np.array([r, I0, f1, Lx, Ly])

dx = 50
dt = 1
T = 2000

res = dt / dx ** 2

# Run simulation
C1_time, C2_time = st.run_model(init_parameters, model_parameters, res, dt)

# Plot results
fig, ax = plt.subplots()

for i, (C1, C2) in enumerate(zip(C1_time, C2_time)):
    if i % 200 == 0:
        im = C1 + C2
        ax.imshow(im, cmap="gray")
        ax.set_title("Time = {} s".format(i*dt))
        plt.pause(0.001)

plt.show()


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
#readdata = functions.read_data_as_array(input)


In [ ]:
dx= nump*umpp
plot_vec_field(readdata[0], 10, 8829, 8073, dx)


NameError: name 'readdata' is not defined

In [ ]:
Phase = readdata[0]
fig, ax = plt.subplots()
title = ax.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=ax.transAxes, ha="center")

# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
def update_fig(i):
    
    sd = 0.1*Phase[i]
    
    error = divide((C_time[i]-Phase[i])**2,sd**2+1)
    error_new = divide((C_time[i]-new_Phase[i])**2,sd**2+1)
    
    im = ax.imshow(error, vmin=0, vmax=1000, animated=True, cmap='Reds')
    if i == 0:
        ax.imshow(error, vmin=0, vmax=1000, cmap='Reds')  # show an initial one first

    title.set_text("time = {} hr".format(i))
    #ims.append([im])

ani = animation.FuncAnimation(fig, func=update_fig, interval=300, frames = len(Phase)-1)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
# writer = animation.FFMpegWriter(
#     fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("movie.mp4", writer=writer)

HTML(ani.to_html5_video())

NameError: name 'readdata' is not defined

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
title1 = ax[0][0].text(0.5,.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},ha="center")
title2 = ax[0][1].text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},ha="center")

cols = ['Error with SD', 'Error without SD']
rows = ['SA with SD', 'SA without SD']

for axis, col in zip(ax[0], cols):
    axis.set_title(col)

for axis, row in zip(ax[:,0], rows):
    axis.set_ylabel(row, size='large')

plt.tight_layout()
# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
def update_fig(i):
    sd_val = 0.1*Phase[i]
    
    im1 = ax[0][0].imshow(divide((Cwsd_time[i]-Phase[i])**2,1+sd**2), vmin=0, vmax=1000, animated=True, cmap='Reds')
    im2 = ax[0][1].imshow((Cwsd_time[i]-Phase[i])**2, vmin=0, vmax=5000, animated=True, cmap='Reds')
    im3 = ax[1][0].imshow(divide((Cwosd_time[i]-Phase[i])**2,1+sd**2), vmin=0, vmax=1000, animated=True, cmap='Reds')
    im4 = ax[1][1].imshow((Cwosd_time[i]-Phase[i])**2, vmin=0, vmax=5000, animated=True, cmap='Reds')
    #if i==0:
    #    fig.colorbar(im1, ax=ax[0])
    #    fig.colorbar(im2, ax=ax[1])
    title1.set_text("time = {} hr".format(i))
    title2.set_text("time = {} hr".format(i))

    #ims.append([im])
    
ani = animation.FuncAnimation(fig, func=update_fig, interval=500, frames = len(Phase)-1)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
# writer = animation.FFMpegWriter(
#     fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("movie.mp4", writer=writer)

HTML(ani.to_html5_video())

NameError: name 'Phase' is not defined

: 

In [28]:
import study_init
study_init.create_circular_mask(8840,8078,center=None,radius=None)
study_init.disk_init(np.array([1, 1, 0.5, 8840, 8078]))

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
functions.model_update(C1, C2, S, A1, A2, Parameters, res, dt)

NameError: name 'C1' is not defined

In [11]:
import numpy as np

ModelParameters = study_init.model_parameters
InitialParameters = study_init.init_parameters

study_init.create_circular_mask(40, 30, center=None, radius=None)
study_init.disk_init(InitialParameters)

im = study_init.run_model(InitialParameters,ModelParameters,study_init.res,study_init.dt)
im
#Image.fromarray(a)

([array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 

In [12]:
import functions as fn
path, fc1, fc2, time_i, time_f, nump, umpp, thresh, dthr, sd_min = 'Feb3_s2t', 'CFP','Cy3',1,17,50,0.91,1,3,0.5 
input = path, fc1, fc2, time_i, time_f, nump, umpp, thresh, dthr, sd_min
fn.read_data_as_array(input)

FileNotFoundError: [Errno 2] No such file or directory: 'Feb3_s2t02_Phase.jpg'